In [1]:
from src.all_in_one import *

No password for tt_pt_FM_v04.11.xlsx


In [2]:
cds = pd.read_csv(r"notebooks/consolidate_datasets/dec24/consolidated_dataset_scr.csv")

In [3]:
# Intake dataset
psint_file = path_manager.get_data_file(Category.PS, PSFile.PSNT)
psint_dataset = Dataset(config_file, psint_file.sheet)
psint = get_df(psint_file.path, psint_file.sheet, config_file).sort_values(by="nt_s1")

In [4]:
psint_0 = psint.drop(columns=["nt_re", "note"]).dropna(
    subset=["nt_s1"]
)  # Intake assessment without re intake
psint_0 = psint_0.drop_duplicates(
    subset=["rid"], keep="first"
)  # This will drop a beneficiary with R0062 - This rid has two beneficiaries!

In [5]:
psreint = psint[["rid", "nt_re"]].dropna()

In [ ]:
# Consolidate Intake and re-intake
cpsint = pd.merge(psint_0, psreint, on="rid", how="left")
reint = cpsint["nt_re"]
cpsint.drop(columns=["nt_re"], inplace=True)
cpsint.insert(9, "nt_re", reint)
cpsint

,intsp,ntspi,rid,fcid,firstname,lastname,nt_s1,nt_s2,nt_s3,nt_re,...,primary_torture,secondary_torture,sexual_violation_a,sexual_violation_p,hrd,journalist,wov,stgbv,lgbti,other
0,YQ,1,R0130,8110020011,صباح,شيخ هلال,2019-04-26,2019-05-28,NaT,2021-01-15,...,Yes,No,NaN,NaN,NaN,NaN,Yes,No,No,No
1,YQ,2,R0024,8110020012,نجوى,حصرم,2019-12-26,2020-01-02,NaT,NaT,...,Yes,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No
2,YQ,16,R0001,8110020026,كوثر,العيسى,2020-01-02,2020-01-09,2020-01-17,NaT,...,Yes,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No
3,YQ,8,R0002,8110020018,عائشة,مرندي,2020-01-03,2020-01-20,NaT,2020-10-28,...,No,Yes,NaN,NaN,NaN,NaN,Yes,No,No,No
4,YQ,5,R0003,8110020015,عبير,خطيب,2020-01-06,2020-01-13,NaT,2021-11-23,...,No,Yes,NaN,NaN,NaN,NaN,Yes,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,LA,26,R1250,8110100025,فاطمة,عاشور,2024-11-14,2024-11-21,NaT,NaT,...,Yes,Yes,Yes,No,No,No,Yes,No,No,No
337,LA,27,R1511,8110100026,دعاء,رزية,2024-11-15,2024-11-26,NaT,NaT,...,No,Yes,No,No,No,No,Yes,Yes,No,No
338,YQ,136,R1345,8110020135,أحمد,قاسم,2024-11-15,2024-11-22,NaT,NaT,...,Yes,Yes,No,No,No,No,No,No,No,No
339,YQ,137,R1358,8110020136,زياد,الصيني,2024-11-26,2024-12-03,NaT,NaT,...,Yes,Yes,No,Yes,No,No,Yes,Yes,No,No


In [7]:
cpsint = cpsint.drop(columns=["firstname", "lastname"], axis=1)

In [8]:
cds_int = pd.merge(cds, cpsint, on="rid", how="left")

In [9]:
cds_int.to_csv(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_psint.csv", index=False
)

In [14]:
# Group Counseling dataset
psg_file = path_manager.get_data_file(Category.PS, PSFile.PSG)
psg_dataset = Dataset(config_file, psg_file.sheet)
psg = get_df(psg_file.path, psg_file.sheet, config_file).drop_duplicates(
    subset=["rid"]
)  # This drop duplicates will drop a beneficiary that has rid of another one.
psg = psg.drop(columns=["note", "firstname", "lastname"], axis=1)


In [15]:
cds_g = pd.merge(cds_int, psg, on="rid", how="left")

In [16]:
cds_g.to_csv(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_psgc.csv", index=False
)

In [18]:
# Individual Counseling dataset
psi_file = path_manager.get_data_file(Category.PS, PSFile.PSI)
psi_dataset = Dataset(config_file, psi_file.sheet)
psi = get_df(psi_file.path, psi_file.sheet, config_file)
psi = get_df(psi_file.path, psi_file.sheet, config_file).drop_duplicates(
    subset=["rid"]
)  # This drop duplicates will drop a beneficiary that has rid of another one.
psi = psi.drop(columns=["note", "firstname", "lastname", "need_pt"], axis=1)

In [19]:
psi = psi.rename(columns={"fcid": "icfcid"})
cds_i = pd.merge(cds_g, psi, on="rid", how="left")

In [20]:
cds_i.to_csv(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_psic.csv", index=False
)

In [21]:
# PS Follow-up dataset
psfu_file = path_manager.get_data_file(Category.PS, PSFile.PSFU)
psfu_dataset = Dataset(config_file, psfu_file.sheet)
psfu = get_df(psfu_file.path, psfu_file.sheet, config_file).drop_duplicates(
    subset=["rid"]
)  # There are two duplicates found.
psfu = psfu.drop(columns=["note", "firstname", "lastname"], axis=1)
psfu = psfu.rename(columns={"fcid": "psfufcid"})

In [22]:
cds_fu = pd.merge(cds_i, psfu, on="rid", how="left")

In [23]:
cds_fu.to_csv(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_psfu.csv", index=False
)